In [1]:
%matplotlib inline
import PI

meta_params = {
    'num_of_train_dataset': 1000,
    'num_of_test_dataset': 100,
    'is_flatten': False, 
    'adv_attack': 'i_FGSM',
    'is_debug': False
}

PI = PI.PIInterface(meta_params)

import torch
import torch.nn as nn
import torch.optim as optim
from MNIST_models import *
model = CNN()
loss_func, optimizer = nn.CrossEntropyLoss(), torch.optim.Adam(model.parameters(), lr=1e-3)
model = PI.train_model(model, loss_func, optimizer, 3)
PI.set_model(model)
print('train acc:', PI.eval_model('train'))
print('test acc:', PI.eval_model('test'))
print()

model = load_model('store/MNIST_CNN.pt')
PI.set_model(model)
print('train acc:', PI.eval_model('train'))
print('test acc:', PI.eval_model('test'))

Generate CNN model, epoch: 1 ...
acc: 0.877
Generate CNN model, epoch: 2 ...
acc: 0.942
Generate CNN model, epoch: 3 ...
acc: 0.963
train acc: 0.963
test acc: 0.89



//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'MNIST_models.CNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dum

train acc: 0.98
test acc: 0.99


In [2]:
PI.generate_LPs()
(B_accuracy, B_TNR), (A_accuracy, A_ASR, A_TNR, A_TPR), (B_LPs, A_LPs), (B_LPs_score, A_LPs_score) = PI.evaluate_algorithm_on_test_set(verbose=True)

KeyError: 'model_type'

In [ ]:
import numpy as np

dropout_rates = [0.05*i for i in range(0, 21)]
# dropout_rates = [0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.9]
savefig = True 
num_of_rob_train_eps = 10
first_fig_prefix_str = 'exp_15_1_'
second_fig_prefix_str = 'exp_15_2_'
ASRs = []

for dropout_rate in dropout_rates:
    print('dropout_rate:', dropout_rate)
    PI.set_dropout_rate(dropout_rate)
    PI.generate_twisted_model('CNN', num_of_rob_train_eps)
    PI.generate_LPs(on_twisted_model=True)    
    
    # Computation 
    (B_accuracy, B_TNR), (A_accuracy, A_ASR, A_TNR, A_TPR), (B_LPs, A_LPs), (B_LPs_score, A_LPs_score) = PI.evaluate_algorithm_on_test_set(True, on_retrained_model=False, on_twisted_model=True)
    ASRs.append(A_ASR)
    print('Attack success rates:', ASRs)
    
    BLPs, ALPs = np.array(B_LPs), np.array(A_LPs) 
    BLPs[BLPs=='benign'] = 1
    BLPs[BLPs=='adversarial'] = 0
    BLPs = BLPs.astype(np.int)
    prob_BLPs = np.sum(BLPs, axis=0) / BLPs.shape[0]
    ALPs[ALPs=='benign'] = 1
    ALPs[ALPs=='adversarial'] = 0
    ALPs = ALPs.astype(np.int)
    prob_ALPs = np.sum(ALPs, axis=0) / ALPs.shape[0]
    results = (prob_ALPs, prob_BLPs, A_LPs_score, B_LPs_score)
    
    # Visualize experimental results 
    draw_exp_results(results, dropout_rate, savefig=savefig, first_fig_prefix_str=first_fig_prefix_str, second_fig_prefix_str=second_fig_prefix_str)


In [ ]:
print(ASRs)

In [ ]:
import matplotlib.pyplot as plt

index = np.arange(len(ASRs))
plt.plot(dropout_rates[:len(ASRs)], ASRs, color='g')
plt.ylim((0,1))
plt.savefig('ASR_vs_dropout')